In [13]:
from faker import Faker
import pandas as pd
import numpy as np
import random
import os
import shutil


In [16]:
fake = Faker('PT-BR')
datas = pd.date_range("2021-01-01", "2025-09-01", freq="D")
# Quantidades
n_usinas = 100
n_regionais = 10
n_clientes = 2000
n_obras = 3000
n_tipos = 4

criando dados fakes

In [28]:
regionais = pd.DataFrame({
    "REGIONAL_ID": range(1, n_regionais+1),
    "NOME": [f"Regional {i}" for i in range(1, n_regionais+1)]
})

usinas = pd.DataFrame({
    "USINA_ID": range(1, n_usinas+1),
    "NOME": [f"Usina {i}" for i in range(1, n_usinas+1)],
    "REGIONAL_ID": np.random.choice(regionais["REGIONAL_ID"], n_usinas)
})

clientes = pd.DataFrame({
    "CLIENTE_ID": range(1, n_clientes+1),
    "NOME": [fake.company() for _ in range(n_clientes)],
    "USINA_ID": np.random.choice(usinas["USINA_ID"], n_clientes)
})

tipo_concreto = pd.DataFrame({
    "TIPOCONCRETO_ID": range(1, n_tipos+1),
    "DESCRICAO": ["Convencional", "Bombeável", "Alto desempenho", "Rápida secagem"]
})

obras = pd.DataFrame({
    "OBRAS_ID": range(1, n_obras+1),
    "CLIENTE_ID": np.random.choice(clientes["CLIENTE_ID"], n_obras),
    "USINA_ID": np.random.choice(usinas["USINA_ID"], n_obras)
})

# --------------------
# Vendas simuladas
# --------------------
vendas = pd.DataFrame({
    "PEDIDO_ID": range(1, 2001),
    "DATA_PEDIDO": np.random.choice(datas, 2000),
    "CLIENTE_ID": np.random.choice(clientes["CLIENTE_ID"], 2000),
    "OBRA_ID": np.random.choice(obras["OBRAS_ID"], 2000),
    "USINA_ID": np.random.choice(usinas["USINA_ID"], 2000),
    "TIPOCONCRETO_ID": np.random.choice(tipo_concreto["TIPOCONCRETO_ID"], 2000),
    "M3_PEDIDO": np.round(np.random.uniform(5, 50, 2000),2),
    "STATUS": np.random.choice(["CONFIRMADO","CANCELADO"], 2000, p=[0.85,0.15]),
    "PRECO_UNITARIO": np.round(np.random.uniform(280,500,2000),2)
})

# --------------------
# Entregas
# --------------------
entregas = vendas[vendas["STATUS"]=="CONFIRMADO"].copy()
entregas = entregas.sample(frac=0.9)  # 90% realizadas

entregas["M3_ENTREGUE"] = entregas["M3_PEDIDO"] * np.random.uniform(0.9,1.05,len(entregas))
entregas["HORA_SAIDA"] = [fake.time() for _ in range(len(entregas))]
entregas["HORA_CHEGADA"] = [fake.time() for _ in range(len(entregas))]
entregas["STATUS"] = np.random.choice(["ENTREGUE","ATRASADO"], len(entregas), p=[0.85,0.15])
entregas["MOTIVO_ATRASO"] = np.where(entregas["STATUS"]=="ATRASADO",
                                     np.random.choice(["Trânsito","Clima","Manutenção"], len(entregas)),
                                     None)

# --------------------
# Contratos
# --------------------
contratos = pd.DataFrame({
    "CONTRATO_ID": range(1, n_clientes+1),
    "ID_OBRA": np.random.choice(obras["OBRAS_ID"], n_clientes),
    "CLIENTE_ID": clientes["CLIENTE_ID"],
    "TIPO_CONCRETO": np.random.choice(tipo_concreto["TIPOCONCRETO_ID"], n_clientes),
    "PRECO": np.round(np.random.uniform(280, 500, n_clientes),2),
    "DESCONTO": np.round(np.random.uniform(0, 0.15, n_clientes),2),
    "M3_CONTRATADO": np.round(np.random.uniform(200, 2000, n_clientes),2),
    "VIGENCIA_INICIO": [fake.date_between(start_date="-3y", end_date="today") for _ in range(n_clientes)],
    "VIGENCIA_FIM": [fake.date_between(start_date="today", end_date="+2y") for _ in range(n_clientes)]
})


In [32]:
# --------------------
# Exporta para CSV
# --------------------
# Diretório de saída
output_dir = "concretdata_csvs"
os.makedirs(output_dir, exist_ok=True)

regionais.to_csv(f"{output_dir}/regionais.csv", index=False, sep=";")
usinas.to_csv(f"{output_dir}/usinas.csv", index=False, sep=";")
clientes.to_csv(f"{output_dir}/clientes.csv", index=False, sep=";")
tipo_concreto.to_csv(f"{output_dir}/tipo_concreto.csv", index=False, sep=";")
obras.to_csv(f"{output_dir}/obras.csv", index=False, sep=";")
vendas.to_csv(f"{output_dir}/vendas.csv", index=False, sep=";")
entregas.to_csv(f"{output_dir}/entregas.csv", index=False, sep=";")
contratos.to_csv(f"{output_dir}/contratos.csv", index=False, sep=";")

#COMPACTA
shutil.make_archive("concretdata_csvs", 'zip', output_dir)
print("✅ Arquivo concretdata_csvs.zip gerado com sucesso!")

✅ Arquivo concretdata_csvs.zip gerado com sucesso!
